In [1]:
%pip install pandas

  Using cached pandas-2.1.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (18 kB)
  Using cached numpy-1.26.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (115 kB)
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.4-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.1.4-cp311-cp311-macosx_11_0_arm64.whl (10.8 MB)
Using cached numpy-1.26.3-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
Using cached pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)
Using cached tzdata-2023.4-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
df = pd.read_csv('data/input/reviews_200.csv')

df = df.dropna()
print(len(df))
df

199


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,3/30/2009,10952,Lam,Daniel is really cool. The place was nice and clean. Very quiet neighborhood. He had maps and a lonely planet guide book in the room for you to use. I didnt have any trouble finding the place from Central Station. I would defintely come back! Thanks!
1,515749,1671407,7/9/2012,2640670,Gregory,"If you want the authentic Amsterdam houseboat experience, this is it! It is a great boat, located on a quiet canal with a view of the Rijksmuseum towers. Spotlessly clean, comfortable bed, hot shower, good wifi signal, nice kitchen with fridge, cofee maker, and stove for cooking meals. The location is very accessible to museums, gym bar and bakery on the corner, nine streets area with lots of resaraunts and bars and jazz clubs, Vondelpark, shopping on Utrechtstraat, and all of the sites of central Amsterdam. \r<br/>\r<br/>Now for the special part: If you are at all interested in boats and maritime history, this one is a great find. The cabin where guests stay is actually the cargo hold within the original hull of a freight boat that once hauled goods all over Holland. (Just imagine the countless tons of cargo that came in and out of the place where you are sleeping!) And yet the interior beautifully and comfortably remodeled using new and salvaged materials including a number of hatches that serve as skylights. The original steerhouse is still intact, along with the original one-cylinder engine down below. But the highlight is the captain's quarters where the family who owned and operated the boat lived during the 1920s (not open for guests to sleep in, yet). Stepping down into the quarters, complete with original cabinets and cookstove, feels like traveling back in time. The host cares a lot about preserving the historical character of his boat, because most other boats like it have long since gone to the scrapyard. He showed us fascinating photographs of the original owners who lived on the boat when it plied the canals nearly 100 years ago. \r<br/>\r<br/>There are many houseboats in Amsterdam. A lot of them are glorified trailer houses on flat barges. This one is a gem."
2,515749,1715674,7/15/2012,1032804,Michael,"Unique and luxurious to be sure. I couldn't recommend staying with Derk any more highly.\r<br/>\r<br/>Coolest thing first: We showed up to Derk's and he actually had three houseboats in a row, and offered to show up the one we saw on the site and another, and let us choose our favorite to stay in! We had an incredible boat with wired internet, a comfortable bed and a very comfortable futon, TV, speakers, hot water, a shower, washer/dryer. I couldn't believe I was on a boat. Plus the charm of being on one was fantastic, and the views out onto the canal were fantastic. It rained one morning and was such a beautiful way to wake up.\r<br/>\r<br/>In addition, Derk was really communicative, responded to our last minute (day before!) request for a reservation quite casually, and met us right at the boat. He even had a few cold beers in the fridge for us, and was really great explaining the area.\r<br/>\r<br/>Would recommend!"
3,2818,1771,4/24/2009,12798,Alice,"Daniel is the most amazing host! His place is extremely clean, and he provides everything you could possibly want (comfy bed, guidebooks & maps, mini-fridge, towels, even toiletries). He is extremely friendly and helpful, and will go out of his way to help you if needed, or just let you come and go as you please. Highly recommended!"
4,515749,1963378,8/12/2012,503786,Brian,"My wife and I recently stopped in Amsterdam for a few days during our honeymoon. We didn't want to stay in the typical Amsterdam hotel and opted for something special - a traditional Dutch houseboat. Needless to say staying on a boat was a special experience, but the boat was very clean, was on a quiet canal in a superb location in central Amsterdam, and had everything we needed for a memorable weekend. The owner was good

In [7]:
%pip install openai tenacity python-dotenv

from dotenv import load_dotenv
from tenacity import retry, stop_after_attempt, wait_random_exponential
from typing import List
import openai
import os

load_dotenv()

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text: str, model, **kwargs) -> List[float]:

    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")
    text = text.replace("<br/>", " ")

    response = openai.embeddings.create(input=[text], model=model, **kwargs)

    return response.data[0].embedding

Note: you may need to restart the kernel to use updated packages.


In [8]:
text="great location!"
model="text-embedding-ada-002"
embedding=get_embedding(text, model)

print("embeddings length: ", len(embedding))
print(embedding)

embeddings length:  1536
[-0.01466985885053873, -0.005897592753171921, 0.002610409865155816, -0.04086419567465782, -0.01060277596116066, 0.01035784836858511, -0.007193129509687424, -0.016912877559661865, 0.020316079258918762, -0.03065459057688713, 0.006049061194062233, 0.030680373311042786, -0.0067870658822357655, -0.014940568245947361, -0.005974938161671162, -0.0033677511382848024, 0.015598004683852196, -0.008449994027614594, 0.002064157510176301, -0.010828367434442043, -0.03230462968349457, 0.012343049049377441, -0.004295896738767624, -0.0356304831802845, 0.004431251436471939, 0.006403561215847731, -0.0038479375652968884, -0.004515042528510094, -0.013458113186061382, 0.0019723097793757915, 0.016874205321073532, -0.006474460940808058, -0.035114847123622894, -0.01644880510866642, -0.008946293964982033, 0.0007476729806512594, -0.0011166753247380257, -0.021128207445144653, 0.01754453219473362, -0.006748393177986145, -0.006087733898311853, -0.013354985974729061, 0.0027183713391423225, 0.0

In [9]:
model="text-embedding-ada-002"
df["embedding"] = df.comments.apply(lambda x: get_embedding(x, model=model))
output_path="data/output/reviews-embeddings.csv"

# remove file if it already exists
if os.path.exists(output_path):
    os.remove(output_path)

df.to_csv(output_path)

In [10]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def search_reviews(df, review_description, n=3, pprint=True):
   embedding = get_embedding(review_description, model='text-embedding-ada-002')
   df['similarities'] = df.embedding.apply(lambda x: cosine_similarity(x, embedding))
   res = df.sort_values('similarities', ascending=False).head(n)
   return res

In [11]:
results = search_reviews(df, "coffee")

results.head()


,listing_id,id,date,reviewer_id,reviewer_name,comments,embedding,similarities
170,785432,7965650,10/10/2013,4638959,Babs,Nice and quite boat in the centre of Amsterdam with all the stuff you need. I really appreciated the coffee and tea who were available. We stayed there with 5 adults (not couples) and some beds were maybe quite small but okay. Daan was very flexible to meet and hand the keys and to help with all special needs from my colleagues :-)...,"[0.004718014970421791, -0.01799982227385044, 0.0060329800471663475, -0.03520670160651207, -0.0170615054666996, 0.03346222639083862, -0.01642714999616146, -0.008649694733321667, -0.0174711924046278, -0.015052714385092258, -0.007116669788956642, 0.01377078890800476, -0.018198058009147644, -0.01616283506155014, -0.01462981104850769, 0.00460237730294466, 0.02245352230966091, -0.016400719061493874, -0.00386229669675231, -0.017894096672534943, -0.027753029018640518, 0.004189386032521725, 0.01589852198958397, 0.009799462743103504, -0.023431487381458282, 0.012621020898222923, 0.021911676973104477, -0.025797102600336075, -0.011041740886867046, -0.014550517313182354, 0.031109824776649475, -0.019704651087522507, -0.006749933119863272, -0.010546151548624039, -0.011510899290442467, -0.008332516998052597, -0.012601196765899658, 0.02229493483901024, 0.01979716122150421, 0.003515383694320917, -0.005653027910739183, -0.011735566891729832, -0.0013133130269125104, -0.007632083259522915, -0.01769586093723774, 0.04007009044289589, 0.009918403811752796, 0.0010638661915436387, -0.022506386041641235, 0.004110091365873814, 0.014801615849137306, 0.03676615655422211, 0.003422873793169856, -0.01190076395869255, -0.008636478334665298, 0.016030678525567055, -0.012535118497908115, -0.002259889617562294, 0.01994253508746624, -0.021383048966526985, -0.004245552700012922, 0.0011951974593102932, -0.02076190896332264, 0.018951354548335075, -0.002405262552201748, -0.01908351108431816, -0.007579220458865166, 0.022466737776994705, 0.014233339577913284, -0.007982299663126469, 0.036237526684999466, 0.011610017158091068, 0.018620962277054787, 0.01679719053208828, 0.005444880109280348, -0.01217829342931509, 0.015409539453685284, 0.01914959028363228, 0.006531873717904091, -0.0009259269572794437, 0.002071565482765436, -0.026682555675506592, -0.03568246588110924, -0.00409687589854002, 0.01926853135228157, 0.03362081199884415, -0.008775243535637856, 0.024607686325907707, -0.021237675100564957, -0.007063806988298893, -0.0023920468520373106, 0.02151520550251007, -0.013103394769132137, 0.0015801055124029517, -0.023841174319386482, -0.004529690835624933, 0.005656331777572632, 0.019334610551595688, -0.01896457001566887, 0.002671229187399149, ...]",0.793069
70,785432,3574220,2/18/2013,2421794,Marat,"Our experience was amazing! In first, the boat is located in perfect place, near subway and moorings of most popular touristic boat lines. It is really center of Amsterdam. Next advantage is facilities on the boat. At morning you can make a cup of coffee and enjoy it on the boat stern looking at canal and nice buildings and bridge. And finally, just in a few hours you'll have great skill of entering inside boat through the small door :-)","[-0.0034154464956372976, -0.022192198783159256, 0.012421594932675362, -0.02732357196509838, -0.02360955998301506, 0.014711682684719563, -0.022612158209085464, -0.004629389848560095, -0.005032944027334452, -0.013084342703223228, -0.01568283699452877, 0.017454538494348526, -0.020013663917779922, -0.012460965663194656, -0.01758577674627304, -0.017021456733345985, 0.026142437011003494, -0.017388921231031418, -0.010781130753457546, -0.027192333713173866, -0.03283553197979927, 0.0082745011895895, 0.012224739417433739, -0.011397944763302803, -0.0170477032661438, 0.017861373722553253, 0.011876960285007954, -0.013845517300069332, -0.02395077794790268, -0.0075133261270821095, 0.021916601806879044, 0.006391248665750027, -0.03351796418428421, -0.01620778627693653, -0.01154886744916439, 0.011260146275162697, 0.

In [12]:
import pandas as pd
import numpy as np
from ast import literal_eval

df_from_file = pd.read_csv('data/output/reviews-embeddings.csv')
df_from_file = df_from_file.dropna()
df_from_file["embedding"] = df_from_file.embedding.apply(literal_eval).apply(np.array)

results = search_reviews(df_from_file, "walkable")
results.head()

,Unnamed: 0,listing_id,id,date,reviewer_id,reviewer_name,comments,embedding,similarities
27,26,515749,4134541.0,4/13/2013,4944625.0,Julie,"The houseboat is located in a quiet,safe, & nice canal. Easy walks to everything you need. I would stay here again if I make it back to Amsterdam.","[0.008445251733064651, -0.008484349586069584, 0.001603848417289555, -0.011820745654404163, -0.015456895343959332, 0.01374959945678711, 0.004994818940758705, -0.012596197426319122, 0.008249759674072266, -0.011279884725809097, -0.011416728608310223, -0.0036003098357468843, -0.006304614711552858, -0.006835701409727335, 0.0055389381013810635, -0.006086315494030714, 0.025570346042513847, -0.031565431505441666, 0.0032989264000207186, -0.02718641236424446, -0.03372887521982193, -0.004594060592353344, 0.00023092485207598656, -0.017151158303022385, -0.009390129707753658, 0.007239717990159988, 0.021451981738209724, -0.03292084112763405, -0.03054887242615223, -0.01887148804962635, 0.020865505561232567, -0.024540754035115242, -0.029115265235304832, -0.02038329280912876, 0.0026603194419294596, -0.017568208277225494, -0.0061775450594723225, 0.02566157467663288, -0.003012205008417368, -0.00305456155911088, -0.007689349353313446, -0.009246768429875374, 0.0055650039575994015, -0.007865292020142078, -0.0029877685010433197, 0.02263796702027321, -0.009220702573657036, 0.003078998066484928, -0.043112486600875854, 0.0010621728142723441, 0.011025745421648026, 0.02791624888777733, -0.011012712493538857, -0.0037338959518820047, -0.0011973881628364325, 0.011266851797699928, -0.004515863489359617, -0.006607627496123314, 0.021438948810100555, -0.01816771738231182, 0.011364597827196121, -0.013117508962750435, -0.038915928453207016, 0.015743616968393326, 0.0010198162635788321, -0.008529964834451675, -0.019184274598956108, 0.0018034131499007344, 0.02274222858250141, -0.0014914405765011907, 0.02885461039841175, -0.010856319218873978, 0.0018115586135536432, 0.024488622322678566, 0.023654524236917496, -0.0051707616075873375, 0.024671081453561783, 0.01264181174337864, 0.030288217589259148, 0.002510442165657878, 0.006425168365240097, -0.020305095240473747, -0.03492789342999458, 0.021191325038671494, 0.03776904195547104, 0.040610190480947495, -0.006770537234842777, 0.022233949974179268, -0.04094904288649559, -0.013241320848464966, -0.004170494619756937, 0.026287149637937546, -0.032842643558979034, -0.0032207295298576355, -0.006952996365725994, -0.01131246704608202, -0.01033500675112009, 0.02760346047580242, -0.0054542249999940395, 0.003909838851541281, ...]",0.820348
6,6,515749,2221365.0,9/7/2012,2804901.0,Anita,"Location is Perfect with-in walking distance of Vondelpark, Van Gogh Museum, Heineken museum, Dam square, Waterlooplein market and of course Jordaan’s charming lanes. (hire a bike from around the corner, travel one side of the city to the other in minutes!) \r<br/>Every corner you turn has something to offer. \r<br/>The boat is moored on a quite, peaceful, picturesque canal. Cafe, restaurants, bars, supermarket and shops all minutes away.\r<br/>Derk is such a nice guy, he was extremely helpful and is so laid back. His home is yours and you feel very welcome.\r<br/>The boat is charming, clean and comfortable (we got a choice of boats, which ever suited us best!) Instant running hot water, wifi, cooker the lot all you need for a comfy stay.\r<br/>We took our 2.5yr old with us and she felt she was in a fairytale, sleeping on a boat, she got up each morning to feed the swans and ducks from the boat, and then later swam naked with all the other toddlers in Vondelpark fountains.\r<br/>We left planing our next stay, so I really can not recommend this holiday anymore, it was perfect. If you want a unique, unforgettable experience then go for this boat.","[0.007200113032013178, -0.01794634945690632, 0.003111284924671054, -0.00642144912853837, -0.020669987425208092, 0.023798126727342606, -0.01071252766996622, -0.026764465495944023, -0.0012910315999761224, -0.022422824054956436, -0.